In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

### Langsmith tracking
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT']=os.getenv('LANGCHAIN_PROJECT')
os.environ['LANGCHAIN_TRACING_V2']="true"

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model= 'gpt-4o')

print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x0000026B192B7D90> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000026B1C0A98A0> root_client=<openai.OpenAI object at 0x0000026B192E7280> root_async_client=<openai.AsyncOpenAI object at 0x0000026B1BBF9F30> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********')


In [3]:
### input and get response form LLM
result= llm.invoke('what is gen ai')

print(result)


content='Generative AI, often abbreviated as Gen AI, refers to a category of artificial intelligence systems designed to generate new content. This content can include text, images, music, and other media, based on the patterns and examples it has been trained on. Unlike traditional AI that might focus on recognizing patterns or making predictions, generative AI creates new data.\n\nExamples of generative AI models include:\n\n1. **Text Generation**: Models like OpenAI’s GPT-3 or GPT-4 can produce human-like text based on prompts given to them. They can assist in writing articles, creating dialogues, or even coding.\n\n2. **Image Generation**: Tools like DALL-E or Stable Diffusion generate images from textual descriptions, enabling users to create artwork, visuals, or concept designs from their imagination.\n\n3. **Music and Audio**: AI systems that compose music or generate sound effects based on user input or learned musical styles.\n\n4. **Video and Animation**: Although more comple

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        ("system","you are an expert AI Enginee. Provide me answers based on questions"),
        ("user", "{input}")
    ]
)

prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='you are an expert AI Enginee. Provide me answers based on questions'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [5]:
## chain
chain = prompt | llm

response= chain.invoke({"input":"can you tell me about langsmith"})
print(response)

content="LangSmith, developed by LangChain, is a suite of tools designed to enhance the building, testing, and evaluation of AI applications. It focuses on helping developers improve the reliability and performance of large language models (LLMs) in real-world settings. LangSmith offers features for tracing, visualizing, and debugging LLM applications, thereby enabling developers to better understand the flow and execution of their projects. It also aids in testing the performance and accuracy of language models, facilitating experimentation with different prompts and configurations to optimize the application's output. Through these capabilities, LangSmith aims to streamline the development process and elevate the quality of AI-driven applications." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 127, 'prompt_tokens': 32, 'total_tokens': 159, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens'

In [6]:
type(response)

langchain_core.messages.ai.AIMessage

In [7]:
## stroutput Parser

from langchain_core.output_parsers import StrOutputParser
output_parser=StrOutputParser()
chain=prompt|llm|output_parser

response=chain.invoke({"input":"Can you tell me about Langsmith?"})
print(response)

Certainly! Langsmith is a platform or tool often associated with facilitating the development and deployment of language models and natural language processing (NLP) applications. While specific details may vary depending on context or updates beyond my last training data in October 2023, platforms like Langsmith typically offer features such as model training, fine-tuning, and integration capabilities.

Langsmith may provide an environment for developers to design, experiment, and implement sophisticated language models, often leveraging frameworks like TensorFlow or PyTorch. It might also include tools for dataset management, model performance evaluation, and possibly pre-trained models that users can adapt for specific tasks such as text generation, sentiment analysis, chatbots, or other AI-driven language-related applications.

If this is a relatively new product or a specific service, further details should be available from official sources or the platform's documentation.


### Simple Gen ai app with Langchain

In [8]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

loader=WebBaseLoader("https://docs.smith.langchain.com/tutorials/Administrators/manage_spend")


docs=loader.load()

### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': '🦜️🛠️ LangSmith', 'language': 'en'}, page_content='🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppPage Not FoundWe could not find what you were looking for.Head back to our main docs page or use the search bar to find the page you need.CommunityTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2025 LangChain, Inc.')]

In [10]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [11]:
vectorstoredb

In [12]:
## Query From a vector db
query="LangSmith has two usage limits: total traces and extended"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppPage Not FoundWe could not find what you were looking for.Head back to our main docs page or use the search bar to find the page you need.CommunityTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2025 LangChain, Inc.'

In [13]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [14]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000026B26FE28F0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000026B274BBD30>, root_client=<openai.OpenAI object at 0x0000026B274BA410>, root_async_client=<openai.AsyncOpenAI object at 0x0000026B26FE3820>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, confi

In [15]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

'LangSmith has two usage limits: one for total traces and one for extended traces. These limits are represented by the two metrics tracked on their usage graph.'

In [16]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [17]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [18]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000026B274BA1A0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | 

In [19]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
response['answer']

'The context does not provide specific information about LangSmith, such as what it is or its features. However, it mentions that LangSmith is associated with LangChain, LangGraph, and various roles and tools like SDKs for Python and JS/TS. If you have any particular questions about LangSmith based on this context, feel free to ask!'

In [20]:
response

{'input': 'LangSmith has two usage limits: total traces and extended',
 'context': [Document(id='017c88bd-75e3-4993-a6db-68106faa4519', metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': '🦜️🛠️ LangSmith', 'language': 'en'}, page_content='🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppPage Not FoundWe could not find what you were looking for.Head back to our main docs page or use the search bar to find the page you need.CommunityTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2025 LangChain, Inc.')],
 'answer': 'The context does not provide specific information about LangSmith, such as what it is or its features. However, it mentions that LangSmith is associated with LangChain, LangGraph, and various roles and tools lik

In [21]:
response['context']

[Document(id='017c88bd-75e3-4993-a6db-68106faa4519', metadata={'source': 'https://docs.smith.langchain.com/tutorials/Administrators/manage_spend', 'title': '🦜️🛠️ LangSmith', 'language': 'en'}, page_content='🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppPage Not FoundWe could not find what you were looking for.Head back to our main docs page or use the search bar to find the page you need.CommunityTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2025 LangChain, Inc.')]